In [14]:
import pandas as pd

metasamples = pd.read_excel("../Metadata3.xlsx")

In [15]:
metasamples.head()

,PATIENT_ID,SAMPLE_ID,SAMPLE_TYPE,GENDER,INCLUDED,DAYS_AFTER_TREATMENT,AGE_DX,VAR1,VAR2,VAR3,...,VAR5,VAR6,VAR7,LANE,FASTA1,FASTA2,ROUND,MSI_ISH,MSI_SCORE,TUMOR_ONLY
0,VHIO_01,1123_3_cfDNA,cfDNA,XX,Y,224.0,55.0,BEFORE,F,PD,...,2L,1L_2L,triplete,1.0,s3://scitron/exomes/1123_1.fastq.gz,s3://scitron/exomes/1123_2.fastq.gz,FIRST,MSS,8.32,No
1,VHIO_01,1261_2_cfDNA,cfDNA,XX,Y,310.0,55.0,NaN,F,PD,...,2L,1L_2L,triplete,1.0,s3://scitron/exomes/1261_1.fastq.gz,s3://scitron/exomes/1261_2.fastq.gz,FIRST,MSS,13.29,No
2,VHIO_01,B19_0971__17_22863_A1__0_gDNA,gDNA,XX,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,s3://scitron/exomes/B19_0971_S4_R1_001.fastq.gz,s3://scitron/exomes/B19_0971_S4_R2_001.fastq.gz,FIRST,NaN,NaN,NaN
3,VHIO_01,B19_0970__VH_17_B_022863_A5__1_tumor,tumor,XX,Y,266.0,55.0,AFTER,F,PD,...,2L,1L_2L,triplete,1.0,s3://scitron/exomes/B19_0970_S3_R1_001.fastq.gz,s3://scitron/exomes/B19_0970_S3_R2_001.fastq.gz,FIRST,MSS,1.95,No
4,VHIO_02,1514_5_cfDNA,cfDNA,XY,Y,336.0,57.0,NaN,F,short SD,...,2L,1L_2L,doblete,1.0,s3://scitron/exomes/1514_1.fastq.gz,s3://scitron/exomes/1514_2.fastq.gz,FIRST,MSS,Na,Yes


In [16]:
from pathlib import Path

PATH = "/Users/jfnavarro/Projects/Scitron/Results/VariantCalling"

# Add tumor only and MSI columns 
tumor_only_values = list()
msi_scores = list()
for index, row in metasamples.iterrows():
    sample_id = row['SAMPLE_ID']
    sample_patient = row['PATIENT_ID']
    tumor_only = not any(metasamples[metasamples['PATIENT_ID'] == sample_patient]['SAMPLE_TYPE'] == 'gDNA')
    germline = all(metasamples[metasamples['PATIENT_ID'] == sample_patient]['SAMPLE_TYPE'] == 'gDNA')
    if row['SAMPLE_TYPE'] == 'gDNA':
        tumor_only_values.append('Na')
    else:
        tumor_only_values.append("Yes" if tumor_only else 'No')
    msi_value = 'Na'
    if not tumor_only and not germline:
        gdna = metasamples[(metasamples['PATIENT_ID'] == sample_patient)
                           & (metasamples['SAMPLE_TYPE'] == 'gDNA')]['SAMPLE_ID'].to_numpy()[0]
        if row['SAMPLE_TYPE'] != 'gDNA':
            for elem in Path(PATH).rglob('*{}_vs_{}_msisensor'.format(sample_id, gdna)):
                with open(elem, 'r') as filehandler:
                    tokens = filehandler.readlines()[1]
                    _,_,msi_value = tokens.split("\t")
    msi_scores.append(msi_value.strip())
metasamples['MSI_SCORE'] = msi_scores
metasamples['TUMOR_ONLY'] = tumor_only_values

In [17]:
metasamples.to_excel('../Metadata3.xlsx', index=False)

In [18]:
metasamples


,PATIENT_ID,SAMPLE_ID,SAMPLE_TYPE,GENDER,INCLUDED,DAYS_AFTER_TREATMENT,AGE_DX,VAR1,VAR2,VAR3,...,VAR5,VAR6,VAR7,LANE,FASTA1,FASTA2,ROUND,MSI_ISH,MSI_SCORE,TUMOR_ONLY
0,VHIO_01,1123_3_cfDNA,cfDNA,XX,Y,224.0,55.0,BEFORE,F,PD,...,2L,1L_2L,triplete,1.0,s3://scitron/exomes/1123_1.fastq.gz,s3://scitron/exomes/1123_2.fastq.gz,FIRST,MSS,8.32,No
1,VHIO_01,1261_2_cfDNA,cfDNA,XX,Y,310.0,55.0,NaN,F,PD,...,2L,1L_2L,triplete,1.0,s3://scitron/exomes/1261_1.fastq.gz,s3://scitron/exomes/1261_2.fastq.gz,FIRST,MSS,13.29,No
2,VHIO_01,B19_0971__17_22863_A1__0_gDNA,gDNA,XX,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,s3://scitron/exomes/B19_0971_S4_R1_001.fastq.gz,s3://scitron/exomes/B19_0971_S4_R2_001.fastq.gz,FIRST,NaN,Na,Na
3,VHIO_01,B19_0970__VH_17_B_022863_A5__1_tumor,tumor,XX,Y,266.0,55.0,AFTER,F,PD,...,2L,1L_2L,triplete,1.0,s3://scitron/exomes/B19_0970_S3_R1_001.fastq.gz,s3://scitron/exomes/B19_0970_S3_R2_001.fastq.gz,FIRST,MSS,1.95,No
4,VHIO_02,1514_5_cfDNA,cfDNA,XY,Y,336.0,57.0,NaN,F,short SD,...,2L,1L_2L,doblete,1.0,s3://scitron/exomes/1514_1.fastq.gz,s3://scitron/exomes/1514_2.fastq.gz,FIRST,MSS,Na,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,VHIO_47,L19_5546_gDNA,gDNA,XY,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,s3://scitron/exomes/L19-5546_CCTCCT_L001_R1_00...,s3://scitron/exomes/L19-5546_CCTCCT_L001_R2_00...,FIRST,NaN,Na,Na
146,VHIO_48,T074_PDX,PDX,XX,N,6.0,88.0,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,s3://scitron/exomes/L19-6485_TAGGAT_L001_R1_00...,s3://scitron/exomes/L19-6485_TAGGAT_L001_R2_00...,THIRD,NaN,43.87,No
147,VHIO_48,L19_5548_gDNA,gDNA,XX,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,s3://scitron/exomes/L19-5548_AACAAC_L001_R1_00...,s3://scitron/exomes/L19-5548_AACAAC_L001_R2_00...,FIRST,NaN,Na,Na
148,VHIO_49,T160_PDX,PDX,XY,N,48.0,61.0,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,s3://scitron/exomes/L19-5555_GAACAG_L008_R1_00...,s3://scitron/exomes/L19-5555_GAACAG_L008_R2_00...,THIRD,NaN,1.61,No
